In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import tqdm # interactive progress bar

plt.style.use('ggplot')
%matplotlib inline

In [2]:
f_names={'MK':'MK', 'SEGMENT':'Сегмент', 'Service':'Услуга', 'Channeltype': 'Тип канала', 'Source': 'Источник ЕИ', 
'football':'Футбол', 'DECLARED_FAULT':'Заявл. неисправность', 'CREATE_DATE':'Дата создания', 'CLOSETIME':'Дата закрытия',
'all_num_of_repeat':'Всего повторок', 'num_of_repeat2':'Повторок за 2 дня', 'num_of_repeat5':'Повторок за 5 дней',
'num_of_repeat10':'Повторок за 10 дней', 'num_of_repeat30':'Повторок за 30 дней', 'num_of_repeat60':'Повторок за 60 дней',
'MITYPE':'Тип МИ', 'MIPRIORITY':'Приоритет МИ', 'MIIMPACT':'Влияние МИ', 'TECHNICAL_DISTRICT':'Техрайон(WFM)',
'NumWFMforEI':'Кол-во WFM', 'Last_WFM_days_waiting':'Кол-во дней ожидания выезда', 'EIDayOfWeek':'День недели заведения ЕИ',
'EIHourCreate':'Час создания ЕИ'}

In [3]:
# Loading old data
url_to_data = 'data/data2908_fromJan2017.csv'
data = pd.read_csv(url_to_data, names=f_names.keys(), na_values='0', delimiter=';', skiprows=[0])
data.shape

C:\Users\MRekunchak\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(362708, 23)

In [4]:
# Loading fresh data
#!pip install pymysql
import pymysql
import pers_data
import sql_queries

mysql_conn= pymysql.connect(host=pers_data.HOST, 
                port=pers_data.PORT,user=pers_data.USER, passwd=pers_data.PASSWD, 
                db=pers_data.DB, charset='utf8')
#connection.use_unicode =  True

data_y=pd.read_sql(sql_queries.SQL_QUERY_TODAY, con=mysql_conn)
data_y.shape

(2176, 23)

In [5]:
# Data preprocessing

## extract categorical and numerical features for OLD data
categorical_columns_x = [c for c in data.columns if data[c].dtype.name == 'object']
numerical_columns_x   = [c for c in data.columns if data[c].dtype.name != 'object']
print('Categorical columns: ', categorical_columns_x)
print('Numerical columns: ', numerical_columns_x)

Categorical columns:  ['MK', 'SEGMENT', 'DECLARED_FAULT', 'CREATE_DATE']
Numerical columns:  ['Service', 'Channeltype', 'Source', 'football', 'CLOSETIME', 'all_num_of_repeat', 'num_of_repeat2', 'num_of_repeat5', 'num_of_repeat10', 'num_of_repeat30', 'num_of_repeat60', 'MITYPE', 'MIPRIORITY', 'MIIMPACT', 'TECHNICAL_DISTRICT', 'NumWFMforEI', 'Last_WFM_days_waiting', 'EIDayOfWeek', 'EIHourCreate']


In [6]:
## extract categorical and numerical features for FRESH data
categorical_columns_y = [c for c in data_y.columns if data_y[c].dtype.name == 'object']
numerical_columns_y   = [c for c in data_y.columns if data_y[c].dtype.name != 'object']
print('Categorical columns: ', categorical_columns_y)
print('Numerical columns: ', numerical_columns_y)

Categorical columns:  ['MK', 'SEGMENT', 'DECLARED_FAULT']
Numerical columns:  ['Service', 'Channeltype', 'Source', 'football', 'CREATE_DATE', 'CLOSETIME', 'all_num_of_repeat', 'num_of_repeat2', 'num_of_repeat5', 'num_of_repeat10', 'num_of_repeat30', 'num_of_repeat60', 'MITYPE', 'MIPRIORITY', 'MIIMPACT', 'TECHNICAL_DISTRICT', 'NumWFMforEI', 'Last_WFM_days_waiting', 'EIDayOfWeek', 'EIHourCreate']


In [7]:
# data[numerical_columns_x].describe()

In [8]:
# data_y[numerical_columns_y].describe()

In [9]:
data[categorical_columns_x].describe() # Stats for categorial features for OLD data

,MK,SEGMENT,DECLARED_FAULT,CREATE_DATE
count,357959,359808,362705,362708
unique,34,8,325,353603
top,ФЛ,МР,"Телефония. Тишина в трубке, нет зуммера",1501132721
freq,277265,287001,32387,8


In [10]:
data_y[categorical_columns_y].describe() # Stats for categorial features for FRESH data

,MK,SEGMENT,DECLARED_FAULT
count,2176,2176,2176
unique,22,7,121
top,ФЛ,МР,ONT не на линии. Проблема с оптическим кабелем
freq,1739,1800,274


In [11]:
## Legends
LEGEND={'Channeltype':{
'9207':'Медная линия',
'9205':'Канальная АТШ',
'9204': 'Канальная PON',
'9986':'Канальная Radio',
'9206':'Кодер видеонаблюдения',
'9636':'IP ОПС',
'9987':'Канальная Радио (юрики)',
'9203':'Канал ADSL',
'9633':'Канальная FTTB',
'9632':'ОПС',
'9202':'Камера видеонаблюдения',
'10769':'Канальная МЕДЬ',
'9197':'RFC PON',
'9198':'RFC АТШ',
'9199':'Интернет PON',
'9200':'Интернет АТШ',
'9201':'Интернет АТШ (юрики)',
'9208':'Телевидение PON',
'9209':'Телевидение АТШ',
'9210':'Телефон PON',
'9211':'Телефон АТШ',
'9212':'УПАТС PON',
'9630':'Телефон FTTB',
'9631':'Телевидение FTTB',
'9634':'Интернет FTTB',
'9635':'RFC FTTB'},
'Service':{
'9650':'Все сервисы',
'9697':'ОПС',
'8677':'Не сетевой инцидент',
'8679':'Телевидение',
'8681':'Технологические сервисы',
'8676':'Дополнительные сервисы',
'8678':'Передача данных',
'8680':'Телефония',
'8682':'Энергетическое и климатическое обеспечение технологических площадок'}}

In [12]:
## Replacement of digital values of categories by their number of repetitions in data
# TODO: need to replace occurence of categories to some more appropriate for expression categories in data
stats_x=dict()
stats_y=dict()

for i in ['MK', 'SEGMENT', 'DECLARED_FAULT', 'Service', 'Channeltype', 'football', 'TECHNICAL_DISTRICT',
          'EIDayOfWeek', 'EIHourCreate']:
    stats_x[i]=data[i].value_counts()
    stats_y[i]=data[i].value_counts() # not data_y!!!!!! Because it's occurrence of categories
    

# print(stats.keys())
# print(stats['Service'])
# stats_x['MK'].values
stats_y['Service'],stats_x['Service']

(8678.0    190792
 8680.0     86218
 9650.0     53148
 8679.0     31927
 8681.0       324
 8677.0       151
 8676.0       140
 8682.0         5
 Name: Service, dtype: int64, 8678.0    190792
 8680.0     86218
 9650.0     53148
 8679.0     31927
 8681.0       324
 8677.0       151
 8676.0       140
 8682.0         5
 Name: Service, dtype: int64)

In [13]:
LEGEND['Channeltype'].keys()

dict_keys(['9207', '9205', '9204', '9986', '9206', '9636', '9987', '9203', '9633', '9632', '9202', '10769', '9197', '9198', '9199', '9200', '9201', '9208', '9209', '9210', '9211', '9212', '9630', '9631', '9634', '9635'])

In [14]:
## Notes for understanding of features
cat_features=set()
for feature in sorted(stats_x.keys()):
    d_first=20
#     print(len(stats[feature]))
    if len(stats_x[feature])< 20:
        d_first = len(stats_x[feature])
    print("\n"+feature+"\n")
    for i in stats_x[feature].keys()[:d_first]:
        if type(i) == str:
            print(stats_x[feature][i], "\t", i)
        else:
            cat_features.add(feature)
# print(cat_features)
if cat_features:
    for feature in cat_features:
        try:
            if LEGEND[feature]:
            
                print("\n", feature, "\n")
                for i in stats_x[feature].keys():
                    print(stats_x[feature][i], "\t", LEGEND[feature][str(int(i))])
        except:
                pass
#     print(stats[feature].keys()[:d_first], stats[feature].values[:d_first])
    


Channeltype


DECLARED_FAULT

32387 	 Телефония. Тишина в трубке, нет зуммера
24629 	 Телефония. Тишина в трубке
21939 	 ONT не на линии. Проблема с оптическим кабелем
21395 	 Интернет. Не работает интернет
16386 	 Интернет. Не открываются все страницы
14795 	 ТВ. Не получен IP-адрес
13618 	 Swap ONT Huawei
13301 	 Не работает ONT. Нет индикации питания.
11749 	 ТВ. Черный или синий экран
10198 	 Не работает канал VPN
10119 	 Интернет. Другое
9334 	 ТВ. Проблемы с пультом
7849 	 ТВ. Рассыпание/замирание
6970 	 SLX_Не работает INTERNET
6950 	 Интернет. Не открываются страницы
6720 	 Телефония. Зуммер занято
6521 	 ТВ. Другое
5980 	 Перенос/замена модели ONT
5813 	 ONT на линии. Высокое затухание
5361 	 Интернет. Нет линка

EIDayOfWeek


EIHourCreate


MK

277265 	 ФЛ
25118 	 SOHO
13658 	 NKA
6820 	 SME
4633 	 GS-SLA4
4012 	 Первая
3992 	 Оператор
3864 	 Выделенное обслуживание (пилот)
3278 	 SME GS
3113 	 Corporate
2087 	 KA
2009 	 GS-SLA1
1509 	 SOHO_GS
1366 	 INDIVIDUAL
1195 	 Corpor

In [15]:
# stats_y['SEGMENT'][:6].plot()
# data.head()
# for i in data_x:
#     print(data[i].describe())

In [16]:
## Cleaning the data

# dropped_data_without_empty_cols = data.dropna(axis=1)
# dropped_data_without_empty_rows = data.dropna(axis=0)
# data.count(axis=0)


for item in ['all_num_of_repeat', 'num_of_repeat2', 'num_of_repeat5', 'num_of_repeat10', 'num_of_repeat30', 'num_of_repeat60']:
    data[item]=data[item][data[item].isin([i for i in range(1000)] + [str(i) for i in range(1000)])]
    data[item]=data[item].replace(to_replace=[np.nan], value=[0])

    data_y[item]=data_y[item][data_y[item].isin([i for i in range(1000)] + [str(i) for i in range(1000)])]
    data_y[item]=data_y[item].replace(to_replace=[np.nan], value=[0])

    

# for col_name in ['SERVICE', 'channeltype', 'source']:
#     data[col_name]=data[col_name].replace(to_replace=[np.nan], value=[0])
#     data[col_name]=data[col_name].replace(to_replace=data[col_name].unique(), value=[i for i in range(len(data[col_name].unique()))])



# data['DECLARED_FAULT']=data['DECLARED_FAULT'].replace(to_replace=data['DECLARED_FAULT'].unique(), value=[i for i in range(1,len(data['DECLARED_FAULT'].unique())+1)]) 

# data['CLIENT_CATEGORY']=data['CLIENT_CATEGORY'][data['CLIENT_CATEGORY'].isin(['ФЛ', 'SOHO', 'NKA', 
# 'Выделенное обслуживание (пилот)', 'GS-SLA4', 'SME', 'Оператор', 'SME GS', 'Corporate', 'KA', 'SOHO_GS', 'GS-SLA1', 
# 'Corporate GS', 'GS-Имидж абоненты', 'Имидж-абоненты (физические лица)', 'GS-SLA3', 'INDIVIDUAL', 'KA GS', 'ФЛ_ИЗ_МРМ',
# 'Развитие', 'СПЕЦ', 'Внутренний', 'Посольства', 'Имидж-абоненты (юридические лица)', 'NKA GS'])]
# data['CLIENT_CATEGORY']=data['CLIENT_CATEGORY'].replace(to_replace=data['CLIENT_CATEGORY'].unique(), value=[i for i in range(1,len(data['CLIENT_CATEGORY'].unique())+1)]) 

# data['SEGMENT']=data['SEGMENT'][data['SEGMENT'].isin(['МР','КОРП','ГОС','Оператор','БС28','Внутренний','БС22'])]
# data['SEGMENT']=data['SEGMENT'].replace(to_replace=data['SEGMENT'].unique(), value=[i for i in range(1,len(data['SEGMENT'].unique())+1)]) 


data['DECLARED_FAULT']=data['DECLARED_FAULT'].map(data.groupby('DECLARED_FAULT').size())
data_y['DECLARED_FAULT']=data_y['DECLARED_FAULT'].map(data_y.groupby('DECLARED_FAULT').size())

data['MK']=data['MK'].map(data.groupby('MK').size())
data_y['MK']=data_y['MK'].map(data_y.groupby('MK').size())

data['SEGMENT']=data['SEGMENT'].map(data.groupby('SEGMENT').size())
data_y['SEGMENT']=data_y['SEGMENT'].map(data_y.groupby('SEGMENT').size())



for i in ['CREATE_DATE', 'CLOSETIME']:
    data[i] = pd.to_numeric(data[i],errors='coerce')
    data_y[i] = pd.to_numeric(data_y[i],errors='coerce')

In [17]:
# data['MK'], data_y['MK']

In [18]:
# data['CREATE_DATE'].unique(), data_y['CREATE_DATE'].unique()

In [19]:
# type(data_y['CREATE_DATE'].head()[1])

In [20]:
## Generation of data
data['decision_time'] = data['CLOSETIME']-data['CREATE_DATE']
data_y['decision_time'] = data_y['CLOSETIME']-data_y['CREATE_DATE']

In [21]:
# data_y['SEGMENT'].head()

In [22]:
## Selecting data
# sel_data = pd.concat((data['CLIENT_CATEGORY'],
# data['SEGMENT'],
# data['SERVICE'],
# data['channeltype'],
# data['source'],
# # data['CHANNELREQUESTID'],
# data['football'],
# data['DECLARED_FAULT'],
# data['all_num_of_repeat'],
# data['num_of_repeat2'],
# data['num_of_repeat5'],
# data['num_of_repeat10'],
# data['num_of_repeat30'],
# data['num_of_repeat60'],
# data['CREATE_DATE'],
# data['CLOSETIME'],
# data['decision_time'],
# # data['TIMELEFTSLA']
#                      ), axis=1)
# sel_data.describe()

In [23]:
print(data_y['football'].unique()[2])
# for i in sel_data['SERVICE']:
#     if i.startswith('2'):
#         pass
#     else:
#         print(i)

6


In [24]:
# # full list of categorical columns, where 'nan' is missing values
# for c in sel_data:
#     print(c, sel_data[c].unique(), len(sel_data[c].unique()))

In [25]:
# # for each numerical/quantitative feature built histogram,
# # for each pair of nemerical built dispersion (scattering) diagram
# from pandas.plotting import scatter_matrix
# scatter_matrix(data, alpha=0.05, figsize=(15, 15));

In [26]:
# data[numerical_columns_x].corr() # matrix of correlations for OLD data
# data_y[numerical_columns_y].corr() # matrix of correlations for FRESH data

In [27]:
## Find the number of completed (not skipped) items
# axis = 0 means that we move counter from top to bottom, and axis = 1 than we move counter from left to right
data.count(axis=0), data_y.count(axis=0)

(MK                       357959
 SEGMENT                  359808
 Service                  362705
 Channeltype              321590
 Source                   359828
 football                 362687
 DECLARED_FAULT           362705
 CREATE_DATE              362706
 CLOSETIME                354402
 all_num_of_repeat        362708
 num_of_repeat2           362708
 num_of_repeat5           362708
 num_of_repeat10          362708
 num_of_repeat30          362708
 num_of_repeat60          362708
 MITYPE                    19710
 MIPRIORITY                14081
 MIIMPACT                  13841
 TECHNICAL_DISTRICT       227050
 NumWFMforEI              227052
 Last_WFM_days_waiting    156997
 EIDayOfWeek              362706
 EIHourCreate             359182
 decision_time            354400
 dtype: int64, MK                       2176
 SEGMENT                  2176
 Service                  2176
 Channeltype              2009
 Source                   2176
 football                 2176
 DECLARE

In [28]:
## fill the missing values
# fill all numerical features (Instead of gaps (missing values), we substitute the median values (data.median(axis=0)))
data = data.fillna(0, axis=0)
data_y = data_y.fillna(0, axis=0)

In [29]:
# before
# data['A1'].describe()

In [30]:
# # fill the missing values in categorical feature with top value (Instead missing values) we substitute 
# # the top value in this column
# data['A1'] = data['A1'].fillna('b')

In [31]:
# result
data_y.describe()

,MK,SEGMENT,Service,Channeltype,Source,football,DECLARED_FAULT,CREATE_DATE,CLOSETIME,all_num_of_repeat,...,num_of_repeat60,MITYPE,MIPRIORITY,MIIMPACT,TECHNICAL_DISTRICT,NumWFMforEI,Last_WFM_days_waiting,EIDayOfWeek,EIHourCreate,decision_time
count,2176.000000,2176.000000,2176.000000,2176.000000,2176.000000,2176.000000,2176.000000,2.176000e+03,2.176000e+03,2176.000000,...,2176.000000,2176.000000,2176.000000,2176.000000,2176.000000,2176.000000,2176.000000,2176.0,2176.000000,2176.000000
mean,1402.216912,1533.951287,8781.398897,8569.186581,8710.108915,2.884191,106.888787,1.504009e+09,4.803717e+08,1.900276,...,1.190717,0.157169,0.195772,0.192096,21.148438,0.614430,-0.813879,3.0,14.653493,3718.231158
std,672.369481,583.870341,298.685745,2486.975451,341.808414,1.284177,101.912374,1.547768e+04,7.013945e+08,2.222493,...,0.854428,0.462004,0.741109,0.732090,18.681001,0.528501,5.704510,0.0,4.296309,8798.430375
min,1.000000,1.000000,8676.000000,0.000000,6026.000000,1.000000,1.000000,1.503954e+09,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-28.000000,3.0,0.000000,0.000000
25%,1739.000000,1800.000000,8678.000000,9204.000000,8750.000000,2.000000,22.500000,1.503996e+09,0.000000e+00,1.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.000000,0.000000
50%,1739.000000,1800.000000,8678.000000,9204.000000,8750.000000,3.000000,67.000000,1.504008e+09,0.000000e+00,1.000000,...,1.000000,0.000000,0.000000,0.000000,27.000000,1.000000,0.000000,3.0,14.000000,0.000000
75%,1739.000000,1800.000000,8680.000000,9205.000000,8750.000000,3.000000,161.000000,1.504021e+09,1.504003e+09,2.000000,...,1.000000,0.000000,0.000000,0.000000,39.000000,1.000000,1.000000,3.0,18.000000,1118.250000
max,1739.000000,1800.000000,9650.000000,10769.000000,10867.000000,13.000000,274.000000,1.504035e+09,1.504035e+09,36.000000,...,15.000000,2.000000,5.000000,4.000000,64.000000,5.000000,2.000000,3.0,22.000000,64205.000000


In [32]:
# try:
#     data_describe = data.describe(include=[object])
#     data_describe
# except:
#     pass

In [33]:
# for c in categorical_columns:
#     data[c] = data[c].fillna(data_describe[c]['top'])
# data.describe(include=[object])

In [34]:
# data['DECLARED_FAULT'].unique()

In [35]:
# # devide categorical (non-numerical) columns to binary and nonbinary
# binary_columns    = [c for c in categorical_columns if data_describe[c]['unique'] == 2]
# nonbinary_columns = [c for c in categorical_columns if data_describe[c]['unique'] > 2]
# print(binary_columns, nonbinary_columns)


In [36]:
# # for non-binary columns|features vectorizing is 1) divide each column to the numbers of unique values in the column
# # 2) and if value exist then only one column's row filled like 1
# # for example, column A4 have 3 unique values: print(data['A4'].unique()) => ['u' 'y' 'l']
# # create 3 new columns: 4_u, A4_y, A4_l
# # If A4 value is 'u', then A4_u = 1, A4_y = 0, A4_l = 0, etc.
# # In Pandas it's performed via using .get_dummies method.

# data_nonbinary = pd.get_dummies(data[nonbinary_columns[6:8]], columns=None) # !!!!!! Limitation 6:10!!!!!!!!!1
# data_nonbinary.columns

In [37]:
# data = data_nonbinary
# data = pd.DataFrame(data, dtype=float)
# print(data.shape)
# print(data.columns)

In [38]:
## Normalization
data_numerical_x = data[numerical_columns_x]
data_numerical_x = (data_numerical_x - data_numerical_x.mean()) / data_numerical_x.std()

data_numerical_y = data_y[numerical_columns_y]
data_numerical_y = (data_numerical_y - data_numerical_y.mean()) / data_numerical_y.std()

# data_numerical_x.describe()
data_numerical_y.describe()

,Service,Channeltype,Source,football,CREATE_DATE,CLOSETIME,all_num_of_repeat,num_of_repeat2,num_of_repeat5,num_of_repeat10,num_of_repeat30,num_of_repeat60,MITYPE,MIPRIORITY,MIIMPACT,TECHNICAL_DISTRICT,NumWFMforEI,Last_WFM_days_waiting,EIDayOfWeek,EIHourCreate
count,2.176000e+03,2.176000e+03,2.176000e+03,2.176000e+03,2.176000e+03,2.176000e+03,2.176000e+03,2.176000e+03,2.176000e+03,2.176000e+03,2.176000e+03,2.176000e+03,2.176000e+03,2.176000e+03,2.176000e+03,2.176000e+03,2.176000e+03,2.176000e+03,0.0,2.176000e+03
mean,5.745251e-16,-3.528632e-16,-9.936011e-16,-1.974830e-15,-1.813173e-12,6.933486e-15,-6.237351e-18,-1.645423e-15,1.280022e-15,1.105529e-15,4.221501e-16,-2.500553e-16,3.891903e-16,-4.180020e-15,-2.549023e-16,-3.298015e-16,-4.898043e-17,6.111329e-16,NaN,-1.145938e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,NaN,1.000000e+00
min,-3.528756e-01,-3.445626e+00,-7.852671e+00,-1.467236e+00,-3.522958e+00,-6.848809e-01,-8.550198e-01,-2.710792e+00,-2.510446e+00,-2.290583e+00,-1.758955e+00,-1.393584e+00,-3.401902e-01,-2.641611e-01,-2.623934e-01,-1.132083e+00,-1.162590e+00,-4.765724e+00,NaN,-3.410717e+00
25%,-3.461796e-01,2.552552e-01,1.167060e-01,-6.885274e-01,-7.791041e-01,-6.848809e-01,-4.050747e-01,1.074992e-01,4.820997e-02,-2.498818e-02,-1.514430e-01,-2.232101e-01,-3.401902e-01,-2.641611e-01,-2.623934e-01,-1.132083e+00,-1.162590e+00,1.426728e-01,NaN,-8.503795e-01
50%,-3.461796e-01,2.552552e-01,1.167060e-01,9.018135e-02,-3.735861e-02,-6.848809e-01,-4.050747e-01,1.074992e-01,4.820997e-02,-2.498818e-02,-1.514430e-01,-2.232101e-01,-3.401902e-01,-2.641611e-01,-2.623934e-01,3.132360e-01,7.295534e-01,1.426728e-01,NaN,-1.521056e-01
75%,-3.394836e-01,2.556573e-01,1.167060e-01,9.018135e-02,8.085530e-01,1.459423e+00,4.487045e-02,1.074992e-01,4.820997e-02,-2.498818e-02,-1.514430e-01,-2.232101e-01,-3.401902e-01,-2.641611e-01,-2.623934e-01,9.555999e-01,7.295534e-01,3.179727e-01,NaN,7.789262e-01
max,2.908077e+00,8.845336e-01,6.310234e+00,7.877269e+00,1.730477e+00,1.459469e+00,1.534301e+01,5.744082e+00,7.724177e+00,6.771797e+00,1.270866e+01,1.616202e+01,3.988779e+00,6.482489e+00,5.201415e+00,2.293858e+00,8.298127e+00,4.932726e-01,NaN,1.709958e+00


In [39]:
# data_numerical_x.head()

In [40]:
## For data
X = data.drop(['all_num_of_repeat','CLOSETIME', 'CREATE_DATE'], axis=1)  
y = data['all_num_of_repeat']
feature_names = X.columns
print(feature_names)
fsel_names={'decision_time':'Время решения'}
for i in feature_names:
    if i!= 'decision_time':
        fsel_names[i]=f_names[i]
# fsel_names

Index(['MK', 'SEGMENT', 'Service', 'Channeltype', 'Source', 'football',
       'DECLARED_FAULT', 'num_of_repeat2', 'num_of_repeat5', 'num_of_repeat10',
       'num_of_repeat30', 'num_of_repeat60', 'MITYPE', 'MIPRIORITY',
       'MIIMPACT', 'TECHNICAL_DISTRICT', 'NumWFMforEI',
       'Last_WFM_days_waiting', 'EIDayOfWeek', 'EIHourCreate',
       'decision_time'],
      dtype='object')


In [41]:
## For data_y
X_y = data_y.drop(['all_num_of_repeat','CLOSETIME', 'CREATE_DATE'], axis=1)  
y_y = data_y['all_num_of_repeat']
feature_names_y = X_y.columns
print(feature_names_y)
fsel_names_y={'decision_time':'Время решения'}
for i in feature_names_y:
    if i!= 'decision_time':
        fsel_names_y[i]=f_names[i]
# fsel_names

Index(['MK', 'SEGMENT', 'Service', 'Channeltype', 'Source', 'football',
       'DECLARED_FAULT', 'num_of_repeat2', 'num_of_repeat5', 'num_of_repeat10',
       'num_of_repeat30', 'num_of_repeat60', 'MITYPE', 'MIPRIORITY',
       'MIIMPACT', 'TECHNICAL_DISTRICT', 'NumWFMforEI',
       'Last_WFM_days_waiting', 'EIDayOfWeek', 'EIHourCreate',
       'decision_time'],
      dtype='object')


In [42]:
print(X.shape)
print(y.shape)
N, d = X.shape
print(X_y.shape)
print(y_y.shape)
N_y, d_y = X_y.shape

(362708, 21)
(362708,)
(2176, 21)
(2176,)


In [43]:
%%time
## Devide for train and test samples for OLD data
from sklearn.model_selection import train_test_split
X_train, X_test_x, y_train, y_test_x = train_test_split(X, y, test_size = 0.0, random_state = 11) # obviously test_size=


## Devide for train and test samples for FRESH data
from sklearn.model_selection import train_test_split
X_train_y, X_test, y_train_y, y_test = train_test_split(X_y, y_y, test_size = 0.9999999999999999, random_state = 11) # obviously test_size=


N_train_y, _ = X_train.shape 
N_test_y,  _ = X_test.shape 
N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print(N_train, N_test)
# print(N_train_y, N_test_y)
# X_train, y_train is a training sample, X_test, y_test is a test sample.

362708 2176
Wall time: 5.53 s


In [44]:
%%time
# Teach the model
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

Wall time: 4min 2s


In [45]:
%%time
y_train_predict = knn.predict(X_train)
y_test_predict = knn.predict(X_test)

err_train = np.mean(y_train != y_train_predict)
err_test  = np.mean(y_test  != y_test_predict)
print(err_train, err_test)

0.391069400179 0.830422794118
Wall time: 2min 30s


In [46]:
# %%time
# from sklearn.model_selection import GridSearchCV
# n_neighbors_array = [1, 3, 5, 7, 10, 15, 20, 40, 60, 235]
# knn = KNeighborsClassifier()
# grid = GridSearchCV(knn, param_grid={'n_neighbors': n_neighbors_array})
# grid.fit(X_train, y_train)

# best_cv_err = 1 - grid.best_score_
# best_n_neighbors = grid.best_estimator_.n_neighbors
# print(best_cv_err, best_n_neighbors)

In [47]:
# %%time
# knn = KNeighborsClassifier(n_neighbors=best_n_neighbors)
# knn.fit(X_train, y_train)

# err_train = np.mean(y_train != knn.predict(X_train))
# err_test  = np.mean(y_test  != knn.predict(X_test))
# print(err_train, err_test)

In [48]:
# %%time
# from sklearn.svm import SVC
# svc = SVC()
# svc.fit(X_train, y_train)

# err_train = np.mean(y_train != svc.predict(X_train))
# err_test  = np.mean(y_test  != svc.predict(X_test))
# print(err_train, err_test)

In [49]:
# %%time
# from sklearn.model_selection import GridSearchCV
# C_array = np.logspace(-3, 3, num=7)
# gamma_array = np.logspace(-5, 2, num=8)
# svc = SVC(kernel='rbf')
# grid = GridSearchCV(svc, param_grid={'C': C_array, 'gamma': gamma_array, 'n'})
# grid.fit(X_train, y_train)
# print('CV error    = ', 1 - grid.best_score_)
# print('best C      = ', grid.best_estimator_.C)
# print('best gamma  = ', grid.best_estimator_.gamma)

In [50]:
# %%time
# svc = SVC(kernel='rbf', C=grid.best_estimator_.C, gamma=grid.best_estimator_.gamma)
# svc.fit(X_train, y_train)

# err_train = np.mean(y_train != svc.predict(X_train))
# # err_test  = np.mean(y_test  != svc.predict(X_test))
# print(err_train, err_test)

In [51]:
# %%time
# from sklearn.model_selection import GridSearchCV
# C_array = np.logspace(-3, 3, num=7)
# svc = SVC(kernel='linear')
# grid = GridSearchCV(svc, param_grid={'C': C_array})
# grid.fit(X_train, y_train)
# print('CV error    = ', 1 - grid.best_score_)
# print('best C      = ', grid.best_estimator_.C)

In [52]:
# %%time
# svc = SVC(kernel='linear', C=grid.best_estimator_.C)
# svc.fit(X_train, y_train)

# err_train = np.mean(y_train != svc.predict(X_train))
# err_test  = np.mean(y_test  != svc.predict(X_test))
# print(err_train, err_test)

In [53]:
# %%time
# from sklearn.grid_search import GridSearchCV
# C_array = np.logspace(-5, 2, num=8)
# gamma_array = np.logspace(-5, 2, num=8)
# degree_array = [2, 3, 4]
# svc = SVC(kernel='poly')
# grid = GridSearchCV(svc, param_grid={'C': C_array, 'gamma': gamma_array, 'degree': degree_array})
# grid.fit(X_train, y_train)
# print('CV error    = ', 1 - grid.best_score_)
# print('best C      = ', grid.best_estimator_.C)
# print('best gamma  = ', grid.best_estimator_.gamma)
# print('best degree = ', grid.best_estimator_.degree)

In [54]:
# %%time
# svc = SVC(kernel='poly', C=grid.best_estimator_.C, 
#           gamma=grid.best_estimator_.gamma, degree=grid.best_estimator_.degree)
# svc.fit(X_train, y_train)

# err_train = np.mean(y_train != svc.predict(X_train))
# err_test  = np.mean(y_test  != svc.predict(X_test))
# print(err_train, err_test)

In [55]:
%%time
from sklearn import ensemble
rf = ensemble.RandomForestClassifier(n_estimators=5, random_state=11)
rf.fit(X_train, y_train)

err_train = np.mean(y_train != rf.predict(X_train))
err_test  = np.mean(y_test  != rf.predict(X_test))
print(err_train, err_test)

MemoryError: 

In [ ]:
## Predict RepeatIncidents with model using and associate data with EI numbers
res=pd.Series(rf.predict(X_test))
# print(X_test.loc[624])
important_items=res.sort_values()[-25:]


important_items_CREATE_DATE=[]
for i in important_items.index:
    important_items_CREATE_DATE.append(data_y.loc[i,'CREATE_DATE'])
#     print(data_y.loc[i,'CREATE_DATE'], data_y.loc[i,'football'], important_items[i])

x=dict()
for i in range(len(important_items)-1):
    x[i]=important_items_CREATE_DATE[i]
mysql_conn= pymysql.connect(host=pers_data.HOST, 
                port=pers_data.PORT,user=pers_data.USER, passwd=pers_data.PASSWD, 
                db=pers_data.DB, charset='utf8')
#connection.use_unicode =  True
result_of_selection=pd.read_sql(sql_queries.SQL_QUERY_FIND_EIs.format(tuple(important_items_CREATE_DATE)), con=mysql_conn)
# for i in result_of_selection[['REQUEST_ID', 'STATUS', 'CREATE_DATE']]:
#     if result_of_selection['CREATE_DATE']==important_items.values:
#         print(result_of_selection['CREATE_DATE'])
# important_items.values
res_of_union=pd.concat([result_of_selection, pd.DataFrame(important_items.values)], axis=1)
res_of_union.rename(columns={0:'RepeatForecast'}, inplace=True)
res_of_union[['REQUEST_ID', 'RepeatForecast', 'CREATE_DATE']].sort_values('RepeatForecast', ascending=False)

In [ ]:
data_y.loc[624,'CREATE_DATE']

In [ ]:
#!pip install sqlalchemy

from sqlalchemy import create_engine
import datetime
# import importlib
# importlib.reload(pers_data)

def write_to_sql(res_of_union, model_name, data_period):
    ## Write to MySQL to new table


    engine = create_engine('mysql+pymysql://'+pers_data.ACCOUNT_FOR_WRITE_TO_SQL)



    res_of_union=res_of_union[['REQUEST_ID', 'RepeatForecast', 'CREATE_DATE']].sort_values('RepeatForecast', ascending=False)
    res_of_union['Model']=model_name
    table_name = 'FORECASTS'+'_'+model_name+'_'+data_period+'_'+str(datetime.datetime.today().month)+'_'+str(datetime.datetime.today().day)
    pd.DataFrame.to_sql(res_of_union, name=table_name, con=engine, if_exists='replace')

In [ ]:
write_to_sql(res_of_union, 'rf', data_period)

In [ ]:
%%time
rfr = ensemble.RandomForestRegressor(n_estimators=15, random_state=11)
rfr.fit(X_train, y_train)

err_train = np.mean(y_train != rfr.predict(X_train))
err_test  = np.mean(y_test  != rfr.predict(X_test))
print(err_train, err_test)

In [ ]:
%%time
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

print("Feature importances:")
for f, idx in enumerate(indices):
    print("{:2d}. feature '{:5s}' ({:.4f})".format(f + 1, feature_names[idx], importances[idx]))

In [ ]:
feature_arr_names = [i for i in fsel_names.values()]
np.array(feature_arr_names)[:d_first]
# d_first

In [ ]:
# %%time

data_period = url_to_data.split('.')[0].split('/')[1][4:]

plt.subplots_adjust(left=0.15)
d_first = 20
plt.figure(figsize=(8, 8)).autofmt_xdate() 
plt.title("Вклад признаков в повторку (по данным за {0})".format(data_period))
plt.bar(range(d_first), importances[indices[:d_first]], align='center')
plt.xticks(range(d_first), np.array(feature_arr_names)[indices[:d_first]], rotation=90)
plt.xlim([-1, d_first])
plt.savefig("FeaturesImportances{0}.pdf".format(data_period))

In [ ]:
%%time
print(len(feature_names))
best_features = indices[:8]
best_features_names = feature_names[best_features]
print(best_features_names)

In [ ]:
%%time
gbt = ensemble.GradientBoostingClassifier(n_estimators=25, random_state=11)
gbt.fit(X_train[best_features_names], y_train)

err_train = np.mean(y_train != gbt.predict(X_train[best_features_names]))
err_test = np.mean(y_test != gbt.predict(X_test[best_features_names]))
print(err_train, err_test)

In [ ]:
## Predict RepeatIncidents with model using and associate data with EI numbers
res=pd.Series(gbt.predict(X_test[best_features_names]))
# print(X_test.loc[624])
important_items=res.sort_values()[-25:]


important_items_CREATE_DATE=[]
for i in important_items.index:
    important_items_CREATE_DATE.append(data_y.loc[i,'CREATE_DATE'])
    print(data_y.loc[i,'CREATE_DATE'], data_y.loc[i,'football'], important_items[i])

x=dict()
for i in range(len(important_items)-1):
    x[i]=important_items_CREATE_DATE[i]
mysql_conn= pymysql.connect(host=pers_data.HOST, 
                port=pers_data.PORT,user=pers_data.USER, passwd=pers_data.PASSWD, 
                db=pers_data.DB, charset='utf8')
#connection.use_unicode =  True
result_of_selection=pd.read_sql(sql_queries.SQL_QUERY_FIND_EIs.format(tuple(important_items_CREATE_DATE)), con=mysql_conn)
# for i in result_of_selection[['REQUEST_ID', 'STATUS', 'CREATE_DATE']]:
#     if result_of_selection['CREATE_DATE']==important_items.values:
#         print(result_of_selection['CREATE_DATE'])
# important_items.values
res_of_union=pd.concat([result_of_selection, pd.DataFrame(important_items.values)], axis=1)
res_of_union.rename(columns={0:'RepeatForecast'}, inplace=True)
res_of_union[['REQUEST_ID', 'RepeatForecast', 'CREATE_DATE']].sort_values('RepeatForecast', ascending=False)

In [ ]:
write_to_sql(res_of_union, 'gbt', data_period)